## kFold train

### 1. 모듈 불러오기

In [13]:
# 디렉토리 경로
import os

# 데이터셋 관련
from detectron2 import model_zoo
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog

# 학습 관련
import copy
import torch

from detectron2.config import get_cfg
import detectron2.data.transforms as T

from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_train_loader, build_detection_test_loader
from detectron2.data import detection_utils as utils

# 기타
import wandb
from detectron2.utils.logger import setup_logger

setup_logger()


# KFold
import json
import numpy as np
from sklearn.model_selection import KFold

### 2. 데이터 등록

#### 리소스 및 경로 등록

In [14]:
# 리소스
coco_dataset_train = 'coco_trash_train'
coco_dataset_test = 'coco_trash_test'

coco_fold_train = 'coco_fold_train'
coco_fold_test = 'coco_fold_test'

# 경로
path_dataset = '/data/ephemeral/home/dataset/'

path_output = './output_fold'
path_output_eval = './output_eval'

#### *수정이 필요한 리소스

In [15]:
path_model_pretrained = 'COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml'
backbone = 'build_resnet_fpn_backbone'

model_title = path_model_pretrained.split("/")[1].split(".")[0]
k = 5

#### k-fold json 파일 생성

In [16]:
def create_kfold_datasets(k, coco_data, output_dir):
    
    image_to_annotations = {}

    for anno in coco_data['annotations']:
        image_id = anno['image_id']

        if image_id not in image_to_annotations:
            image_to_annotations[image_id] = []

        image_to_annotations[image_id].append(anno)
    
    image_ids = list(image_to_annotations.keys())
    kfold = KFold(n_splits=k, shuffle=True, random_state=22)

    for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(image_ids)):
        train_image_ids = [image_ids[i] for i in train_idx]
        val_image_ids = [image_ids[i] for i in val_idx]

        # Train/Val annotation, 이미지 필터링
        train_annotations = [anno for image_id in train_image_ids for anno in image_to_annotations[image_id]]
        val_annotations = [anno for image_id in val_image_ids for anno in image_to_annotations[image_id]]

        train_images = [img for img in coco_data['images'] if img['id'] in train_image_ids]
        val_images = [img for img in coco_data['images'] if img['id'] in val_image_ids]

        
        # train fold
        train_data = coco_data.copy()
        train_data['annotations'] = train_annotations
        train_data['images'] = train_images

        with open(os.path.join(output_dir, f'train_fold_{fold_idx}.json'), 'w') as f:
            json.dump(train_data, f)


        # validation fold
        val_data = coco_data.copy()
        val_data['annotations'] = val_annotations
        val_data['images'] = val_images
        
        with open(os.path.join(output_dir, f'val_fold_{fold_idx}.json'), 'w') as f:
            json.dump(val_data, f)

#### 데이터셋 등록

In [17]:
def register_kfold_datasets(k, path_dataset):
    for fold_idx in range(k):
        train_dataset_name = f'{coco_fold_train}{fold_idx}'
        val_dataset_name = f'{coco_fold_test}{fold_idx}'

        train_json = os.path.join(path_dataset, f'train_fold_{fold_idx}.json')
        val_json = os.path.join(path_dataset, f'val_fold_{fold_idx}.json')

        if train_dataset_name not in DatasetCatalog.list():
            register_coco_instances(train_dataset_name, {}, train_json, path_dataset)
        
        if val_dataset_name not in DatasetCatalog.list():
            register_coco_instances(val_dataset_name, {}, val_json, path_dataset)

### 3. 초기 설정

#### 기본 설정

In [ ]:
from datetime import datetime

wandb.init(project="2024 부스트캠프 재활용품 분류대회(22, CSV)", 
           name=f'{model_title} {datetime.now().strftime("%m-%d %H:%M")}')

In [19]:
cfg = get_cfg()

# COCO 사전 학습 가중치 경로 설정
cfg.merge_from_file(model_zoo.get_config_file(path_model_pretrained))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(path_model_pretrained)

cfg.SEED = 22
cfg.DATALOADER.NUM_WOREKRS = 4

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10                     # 분류할 클래스 수
cfg.TEST.EVAL_PERIOD = 500                               # 검증 주기(단위 : iter)

schedulers = ['WarmupMultiStepLR',]

#### *하이퍼파라미터

In [20]:
cfg.SOLVER.IMS_PER_BATCH = 4                              # 배치크기
cfg.SOLVER.BASE_LR = 0.0005                               # 초기 학습률
cfg.SOLVER.MAX_ITER = 20000                               # 최대 학습 반복 수
cfg.SOLVER.STEPS = (cfg.SOLVER.MAX_ITER // 2, 
                    cfg.SOLVER.MAX_ITER * 2 //3)          # 학습률 감소 단계(50%, 75%) 권장

cfg.SOLVER.GAMMA = 0.05                                   # 학습률 감소 비율(%)
cfg.SOLVER.AMP.ENABLED = True
cfg.SOLVER.LR_SCHEDULER_NAME = schedulers[0]

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128            # 각 이미지당 샘플링할 RoI 개수

cfg.MODEL.RPN.NMS_THRESH = 0.7

cfg.MODEL.BACKBONE.NAME = backbone
cfg.MODEL.FPN.IN_FEATURES = ["res2", "res3", "res4", "res5"]
cfg.MODEL.RPN.IN_FEATURES = ["p2", "p3", "p4", "p5"]


cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[16, 32, 64, 128, 256, 512]]
# cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.5, 1.0, 2.0]]
# cfg.MODEL.ANCHOR_GENERATOR.ANGLES = [[-90, 0, 90]]


aug_list = [                                            # 데이터 증강 옵션
    T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
    T.RandomBrightness(0.8, 1.8),
    T.RandomContrast(0.6, 1.3),
    T.RandomRotation(angle=[-15, 15]),
    T.RandomCrop(crop_type="relative_range", crop_size=(0.8, 0.8)),
    T.RandomLighting(0.1),
]

config = {
    "model": model_title,
    "backbone": backbone,
    "roi size": cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE,
    "learning rate": cfg.SOLVER.BASE_LR,
    "scheduler": cfg.SOLVER.LR_SCHEDULER_NAME,
    "augmentation": aug_list,
}
wandb.config.update(config)

### 4. 학습

#### 전처리 및 증강

In [21]:
def MyMapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)
    image = utils.read_image(dataset_dict['file_name'], format='BGR')
    
    transform_list = aug_list

    image, transforms = T.apply_transform_gens(transform_list, image)
    
    dataset_dict['image'] = torch.as_tensor(image.transpose(2,0,1).astype('float32'))
    
    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop('annotations')
        if obj.get('iscrowd', 0) == 0
    ]
    
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict['instances'] = utils.filter_empty_instances(instances)
    
    return dataset_dict

#### Trainer

In [22]:
class MyTrainer(DefaultTrainer):
    
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=MyMapper)
    
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
            os.makedirs(output_folder, exist_ok=True)

        return COCOEvaluator(dataset_name, cfg, False, output_folder)
    
    
    # Wandb 기록용
    def log_metrics(self, results):
        wandb.log({
            "bbox_AP": results["bbox"]["AP"],
            "bbox_AP50": results["bbox"]["AP50"],
            "bbox_AP75": results["bbox"]["AP75"],
            "bbox_IoU": results["bbox"]["AP"],
        })


    # Wandb 기록용
    def evaluate_model(self):
        evaluator = COCOEvaluator(coco_dataset_test, self.cfg, False, output_dir=path_output_eval)
        val_loader = build_detection_test_loader(self.cfg, coco_dataset_test)
        results = inference_on_dataset(self.build_model(self.cfg), val_loader, evaluator)
        self.log_metrics(results)

#### 학습

In [23]:
def kfold_training(k, path_dataset, cfg):
    for fold_idx in range(k):
        cfg.DATASETS.TRAIN = (f'{coco_fold_train}{fold_idx}',)
        cfg.DATASETS.TEST = (f'{coco_fold_test}{fold_idx}',)
        
        cfg.OUTPUT_DIR = f'{path_output}_{fold_idx}'
        os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

        trainer = MyTrainer(cfg)
        trainer.resume_or_load(resume=False)
        trainer.train()

        trainer.evaluate_model()

In [ ]:
with open(os.path.join(path_dataset, 'train.json'), 'r') as f:
    coco_data = json.load(f)

create_kfold_datasets(k, coco_data, path_dataset)
register_kfold_datasets(k, path_dataset)
kfold_training(k, path_dataset, cfg)

wandb.finish